In [1]:
import os
import glob
from tifffile import imread, imwrite
from vollseg import Augmentation2D
from pathlib import Path
import numpy as np
from scipy.ndimage import gaussian_filter

PyDIPjavaio unavailable:
DLL load failed while importing PyDIPjavaio: The specified module could not be found.

libjvm not found
DIPlib -- a quantitative image analysis library
Version 3.1.0 (Sep 24 2021)
For more information see https://diplib.org


# The size of the images in the directory should be the same else concatenation of arrays would nto work

In the cell below we duplicate the image pixels in XY to a desired size for creating optimal training patch

In [2]:
ImageDir =  '/Users/rando/Downloads/TrainingDatasets/Human_breast_cells/Raw/'
LabelDir = '/Users/rando/Downloads/TrainingDatasets/Human_breast_cells/BinaryMask/'

Aug_image_dir =  '/Users/rando/Downloads/TrainingDatasets/Human_breast_cells/AugRaw/'
Aug_label_dir = '/Users/rando/Downloads/TrainingDatasets/Human_breast_cells/AugBinaryMask/'

Path(Aug_image_dir).mkdir(exist_ok=True)
Path(Aug_label_dir).mkdir(exist_ok=True)


All the parameters that would be needed are provided in the cell below

In [3]:
#All choices below are optional
size = [800,800]
gauss_filter_size = 0
#choices for augmentation below are 1 or 2 or both or None
flip_axis= (1,2)
shift_axis= (1,2)
zoom_axis= (1,2)
#shift range can be between -1 and 1, zoom range > 0
shift_range= (-1,1)
zoom_range= 2
rotate_axis= (1,2)
rotate_angle='random' 
#if zero padding also neds to be done
size_zero = [1000,1000]

In [4]:
Raw_path = os.path.join(ImageDir, '*tif')
filesRaw = glob.glob(Raw_path)
filesRaw.sort

Label_path = os.path.join(LabelDir, '*tif')
filesLabel = glob.glob(Label_path)
filesLabel.sort

Data = []
Label = []
for fname in filesRaw:

    for secondfname in filesLabel:

        Name = os.path.basename(os.path.splitext(fname)[0])
        LabelName = os.path.basename(os.path.splitext(secondfname)[0])
        if Name == LabelName:
            image = imread(fname)
            if image.shape == (38,512,512):
                Data.append(image[11,:,:])
                labelimage = gaussian_filter(imread(secondfname), gauss_filter_size)
                Label.append(labelimage[11,:,:])   
            
Data = np.asarray(Data)
Label = np.asarray(Label)

First perform the extend pixels to a certain size and zero padding operations

In [5]:
extend_pixels = Augmentation2D(size=size)
aug_extend_pixels = extend_pixels.build(data=Data, label=Label, batch_size = Data.shape[0])
aug_extend_pixels_pair = np.asarray(next(aug_extend_pixels))
count = 0
for i in range(0, aug_extend_pixels_pair.shape[1]):
     Name = 'aug_extend_pixels' + str(count)
     imwrite(Aug_image_dir + '/' + Name + '.tif', aug_extend_pixels_pair[0,i,:,:].astype('float32'))
     imwrite(Aug_label_dir + '/' + Name + '.tif', aug_extend_pixels_pair[1,i,:,:].astype('uint16'))
     count = count + 1   

In [6]:
embed_pixels = Augmentation2D(size_zero=size_zero)
aug_embed_pixels = embed_pixels.build(data=Data, label=Label, batch_size = Data.shape[0])
aug_embed_pixels_pair = np.asarray(next(aug_embed_pixels))
count = 0
for i in range(0, aug_embed_pixels_pair.shape[1]):
     Name = 'aug_embed_pixels' + str(count)
     imwrite(Aug_image_dir + '/' + Name + '.tif', aug_embed_pixels_pair[0,i,:,:].astype('float32'))
     imwrite(Aug_label_dir + '/' + Name + '.tif', aug_embed_pixels_pair[1,i,:,:].astype('uint16'))
     count = count + 1   